In [4]:
import time 
import pandas as pd 
import requests
from collections import defaultdict
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
website = 'https://tradelink.com.au/store-finder'

In [3]:
options = Options()
options.add_argument("start-maximized")
options.add_argument("--disable-infobars")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"
    )
options.add_argument("--disable-cookies")

In [ ]:
def populate(contents):
    columns = ['trade-class', 'address', 'latitued', 'longitude', 'email', 'phone', 'branch hours', 'showroom hours']
    for content, column in zip(contents, columns):
        data[column].append(content)

def get_latlng(address):
    base_url = 'https://nominatim.openstreetmap.org/search'
    try:
        response = requests.get(f"{base_url}?q={address}&format=json") 
        response_json = response.json()[0]
        lat, lon = response_json['lat'], response_json['lon']
        return lat, lon
    except:
        return '', ''
        
def get_contacts():
    email = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'SLM_Store_Header_Email'))
    ).text.split(': ')[1]
    phone = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'SLM_Store_Header_Phone_'))
    ).text.split(': ')[1]
    return email, phone

def get_hours():
    elements = driver.find_elements(By.XPATH, '//ul[contains(@class, "SLM_Store_Hours SLM_Store_Timetable_Column")]')
    try:
        branch_hours = elements[0].text.split('\n')[1:]
    except:
        branch_hours = 'Not available'
    try:
        showroom_hours = elements[1].text.split('\n')[1:]
    except:
        showroom_hours = 'Not available'
        
    return branch_hours, showroom_hours


def get_directions():
    class_name = 'SLM_Store_Header_Address'
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, class_name))
    )
    address = element.text
    try:
        latitude, longitude = get_latlng(address)
        return address, latitude, longitude
    except:
        return address,  '', ''
     
def get_trade_classification():
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "SLM_Store_Header_Title"))
    )
    return element.text

def open_in_newtab(url):
    return driver.execute_script(f"window.open('{url}', '_blank');")
    
def go_to_newtab(index):
    return driver.switch_to.window(driver.window_handles[index])
    
if __name__ == '__main__':
    data = defaultdict(list)
    driver = webdriver.Chrome(options = options)
    driver.get(website)
    time.sleep(2)
    elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'hyperlink'))
    )
    urls = [element.get_attribute('href') for element in elements]
    cnt = 0
    for url in urls:
        cnt += 1
        open_in_newtab(url)
        time.sleep(2)
        go_to_newtab(1)
        time.sleep(2)
        classification = get_trade_classification()
        time.sleep(2)
        email, phone = get_contacts()
        time.sleep(2)
        branch_hours, showroom_hours = get_hours()
        time.sleep(2)
        address, latitude, longitude = get_directions()
        time.sleep(2)
        populate([classification, address, latitude, longitude, email, phone, branch_hours, showroom_hours])
        driver.close()
        go_to_newtab(0)
        time.sleep(3)
        if cnt%10 == 0:
            print(cnt)
        
            
        # print(classification, address, zip_code, latitude, longitude, email, phone)
    

In [ ]:
df = pd.DataFrame(data)
df.head()

In [ ]:
df.to_excel('tradelink.xlsx', index=False)

In [10]:
import requests

url = 'https://www.latlong.net/_spm4.php'
data = {
    "c1": "164 Queen Street, Warragul VIC 3820",
    "action": "gpcm",
    "cp": ""    
}

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Referer": "https://www.latlong.net/convert-address-to-lat-long.html",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
}

try:
    response = requests.post(url, data=data, headers=headers)
    response.raise_for_status()  # Raise an exception for 4xx and 5xx status codes
    print("Request successful")
    print("Response:", response.text)
except requests.exceptions.HTTPError as err:
    print("HTTP error occurred:", err)
except Exception as err:
    print("An error occurred:", err)


Request successful
Response: 


In [14]:
import requests
r = requests.post('http://text-processing.com/api/phrases/',
                  data= {'text':'This is California.'})

In [15]:
r.content

b'{"NP": ["California", "This"], "GPE": ["California"], "VP": ["is"], "LOCATION": ["California"]}'